# LangChain: Q&A over Documents
An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
%pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [4]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
#%pip install docarray

In [5]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [6]:
query ="Please list all your jeans \
in a table in markdown and summarize each one."

In [7]:
response = index.query(query)

In [8]:
display(Markdown(response))



| ProductID | ProductName | ProductBrand | Gender | Price (INR) | NumImages | Description | PrimaryColor |
|-----------|------------|--------------|-------|-------------|-----------|-------------|--------------|
| 1009547  | Slub Blue Washed Slim Jeans | Slub | Men | 1999 | 6 | A pair of blue 5-pocket mid-rise jeans, lightly washed and lightly distressed, has whiskers and chevrons, a button fly closure, and a waistband with belt loops | Blue |
| 10145017 | GAP Girls Super Skinny Jeans with Fantastiflex | GAP | Girls | 1149 | 3 | Our tried-and-true classic Premium stretch 1969 denimDark indigo wash.Button at center front (sizes 4-8 have a hook & bar closure beneath faux snap for easy on and off), zip fly.Five-pocket styling.The water used in the process of washing and dyeing these jeans is specially treated to ensure it is safe and clean when it leaves the factory. |  |
| 10003569 | Gini and Jony Girls Blue Regular Fit Mid-Rise Clean Look Stretchable Jeans | Gini and Jony | Girls

In [9]:
loader = CSVLoader(file_path=file)

In [10]:
docs = loader.load()

In [11]:
docs[0]

Document(page_content='ProductID: 10017413\nProductName: DKNY Unisex Black & Grey Printed Medium Trolley Bag\nProductBrand: DKNY\nGender: Unisex\nPrice (INR): 11745\nNumImages: 7\nDescription: Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer\nPrimaryColor: Black', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [12]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [13]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [14]:
print(len(embed))

1536


In [15]:
print(embed[:5])

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [20]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [21]:
query = "Please suggest a shirt with sunblocking"

In [22]:
docs = db.similarity_search(query)

In [23]:
len(docs)

4

In [24]:
docs[0]

Document(page_content="ProductID: 10205415\nProductName: GAP Men's Vintage Slub Jersey Colorblock T-Shirt\nProductBrand: GAP\nGender: Men\nPrice (INR): 749\nNumImages: 5\nDescription: Garment-dyed and washed with a special technique for incredible softness so they already feel like a favoriteMade to be your new everywhere, always-on essentialSoft, airy, textured slub cotton jerseyShort sleevesRib-knit trim at crewneckFront and rear yoke seam detailingColorblock styling throughout\nPrimaryColor: ", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 8678})

In [25]:
retriever = db.as_retriever()

In [26]:
llm = ChatOpenAI(temperature = 0.0)

In [40]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))]) # this does it manually


In [44]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [ ]:
display(Markdown(response))

In [27]:
# the above can be encapsulated in this code below
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [31]:
query =  "Please list all jeans in a table \
in markdown and summarize each one."

In [32]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
display(Markdown(response))

| ProductID | ProductName | ProductBrand | Gender | Price (INR) | NumImages | Description | PrimaryColor |
|-----------|-------------|--------------|--------|-------------|-----------|-------------|--------------|
| 10145017  | GAP Girls Super Skinny Jeans with Fantastiflex | GAP | Girls | 1149 | 3 | Our tried-and-true classic Premium stretch 1969 denimDark indigo wash.Button at center front (sizes 4-8 have a hook & bar closure beneath faux snap for easy on and off), zip fly.Five-pocket styling.The water used in the process of washing and dyeing these jeans is specially treated to ensure it is safe and clean when it leaves the factory. | - |
| 10145033  | GAP Girls Superdenim Jeggings with Fantastiflex | GAP | Girls | 899 | 3 | Super stretchy and super soft-for everyday-awesome, saving-the-world type of adventuresPremium 1969 denim with high stretchDark indigo wash with fading and whiskeringElasticized waist, faux flyFaux front curve pockets, back patch pocketsThe water used in the process of washing and dyeing these jeans is specially treated to ensure it is safe and clean when it leaves the factory. | - |
| 10180489  | GAP Boys Superdenim Slim Jeans with Fantastiflex | GAP | Boys | 1249 | 4 | Super stretchy and super soft- for everyday-awesome, saving-the-world type of adventuresPremium 1969 denim with high stretchMedium indigo wash with fading and whiskeringButton at center front (sizes 4-8 have a hook & bar closure beneath faux snap for easy on and off), zip flyFive-pocket stylingThe water used in the process of washing and dyeing these jeans is specially treated to ensure it is safe and clean when it leaves the factory | - |
| 10144959  | GAP Boys Superdenim Slim Jeans With Fantastiflex | GAP | Boys | 1999 | 3 | Super stretchy and super soft-for everyday-awesome, saving-the-world type of adventuresPremium 1969 denim with high stretchDark indigo washHook & bar closure beneath faux snap for easy on and off, zip flyFive-pocket stylingThe water used in the process of washing and dyeing these jeans is specially treated to ensure it is safe and clean when it leaves the factory | - |

Summary:
1. GAP Girls Super Skinny Jeans with Fantastiflex: Classic premium stretch denim jeans for girls with a dark indigo wash. Features a button or hook & bar closure, zip fly, and five-pocket styling. The water used in the manufacturing process is treated to ensure safety and cleanliness.
2. GAP Girls Superdenim Jeggings with Fantastiflex: Super stretchy and soft jeggings for girls, perfect for everyday adventures. Made with premium denim, these jeggings have a dark indigo wash with fading and whiskering. They have an elasticized waist, faux fly, and faux front curve pockets. The water used in the manufacturing process is treated to ensure safety and cleanliness.
3. GAP Boys Superdenim Slim Jeans with Fantastiflex: Super stretchy and soft slim jeans for boys, designed for everyday adventures. Made with premium denim, these jeans have a medium indigo wash with fading and whiskering. They feature a button or hook & bar closure, zip fly, and five-pocket styling. The water used in the manufacturing process is treated to ensure safety and cleanliness.
4. GAP Boys Superdenim Slim Jeans With Fantastiflex: Super stretchy and soft slim jeans for boys, perfect for everyday adventures. Made with premium denim, these jeans have a dark indigo wash. They have a hook & bar closure, zip fly, and five-pocket styling. The water used in the manufacturing process is treated to ensure safety and cleanliness.

In [34]:
response = index.query(query, llm=llm) # this does everything in one line

In [ ]:

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])